# Day17
BERT多任务微调实战，使用bert-large-uncased。[模型信息](https://huggingface.co/google-bert/bert-large-uncased)
  - 在GLUE benchmark上进行多任务微调(MNLI/QQP/SST-2)
  - 实现任务间的知识迁移和参数共享
  - 对比单任务vs多任务性能
  - 目标：MNLI准确率≥82%，QQP F1≥85%，SST-2准确率≥90%
  
 * 基础设施搭建
    - 多任务数据加载与预处理
    - 共享编码器与多任务头设计
    - 实现动态任务采样器
  
  * 多任务训练策略
    - PCGrad解决任务冲突
    - 梯度累积与归一化
  
  * 实验任务
    - 分类：SST-2情感分析
    - 匹配：QQP语义相似度
    - 推理：MNLI自然语言推理
  
  * 训练与评估目标
    - SST-2：准确率 ≥ 92%
    - QQP：F1 ≥ 85%
    - MNLI：准确率 ≥ 82%

多任务学习（Multi-task Learning - MTL）的核心在于：
- 参数共享 (Parameter Sharing): 大部分参数（这里是强大的 BERT 编码器）在多个任务之间共享。这意味着模型学习到的特征表示是通用的，对多个任务都有用。
- 知识迁移 (Knowledge Transfer): 通过同时训练多个相关任务，一个任务中学到的知识可以帮助改善其他任务的学习。不同的任务可以提供互补的信息，相当于给模型更多元的“视角”来理解数据。这有助于模型学习到更鲁棒、泛化能力更好的特征表示，并可能在单个任务上表现更好（特别是对于数据量较少的任务）。

In [10]:
from datasets import load_dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from transformers import AutoModel
import evaluate
from transformers import DataCollatorWithPadding
from tqdm import tqdm

In [2]:
mnli = load_dataset("glue", "mnli")
sst2 = load_dataset("glue", "sst2")
qqp = load_dataset("glue", "qqp")

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

test_matched-00000-of-00001.parquet:   0%|          | 0.00/1.22M [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/3.73M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

任务特定预处理： 对每个数据集应用 BERT Tokenizer 进行分词、转换为 ID 序列。关键点： 不同任务的输入格式不同！
- SST-2: 单句子输入。[CLS] sentence [SEP]
- QQP: 句子对输入 (判断语义是否相同)。[CLS] sentence1 [SEP] sentence2 [SEP]
- MNLI: 句子对输入 (判断蕴含关系)。[CLS] premise [SEP] hypothesis [SEP]

In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased",use_fast=True)
def tokenize_sst2(examples):
    return tokenizer(
        examples["sentence"],
        padding="max_length",
        truncation=True
    )
def tokenize_mnli(examples):
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        padding="max_length",
        truncation=True
    )
def tokenize_qqp(examples):
    return tokenizer(
        examples["question1"],
        examples["question2"],
        padding="max_length",
        truncation=True
    )
# 如果设置return_tensors="pt"，会返回tensor
# 但Huggingface Datasets 的 .map 期望返回 numpy 数组或 list
# 如果map()输入tensor能会会报错或兼容性不好。
# 之后交给 DataLoader，由 collate_fn 去转成 tensor。

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [26]:
task_to_id = {"mnli": 0, "qqp": 1, "sst2": 2}

def preprocess_function(examples, task_name):
    """
    按照不同数据集的预处理要求分词，同时标记数据集类型
    examples 单个数据集
    task_name 数据集名称
    """
    if task_name == "sst2":
        tokenized_inputs = tokenize_sst2(examples)
        tokenized_inputs["labels"] = examples["label"]
    elif task_name == "mnli":
        tokenized_inputs = tokenize_mnli(examples)
        tokenized_inputs["labels"] = examples["label"]
    elif task_name == "qqp":
        tokenized_inputs = tokenize_qqp(examples)
        tokenized_inputs["labels"] = examples["label"]
    else:
        raise ValueError("Unknown task name")
    # 标记输入属于哪个数据集，在forward中需要这些字段
    tokenized_inputs["task_id"] = [task_to_id[task_name]] * len(tokenized_inputs["input_ids"])
    return tokenized_inputs

# map(function)
processed_sst2 = sst2.map(lambda examples: preprocess_function(examples, "sst2"),
                          batched=True, 
                          remove_columns=["sentence"],
                          desc="Tokenizing sst2")
processed_mnli = mnli.map(lambda examples: preprocess_function(examples, "mnli"), 
                          batched=True, 
                          remove_columns=["premise","hypothesis"],
                          desc="Tokenizing mnli")
processed_qqp = qqp.map(lambda examples: preprocess_function(examples, "qqp"), 
                        batched=True, 
                        remove_columns=["question1","question2"],
                        desc="Tokenizing qqp")


Tokenizing sst2:   0%|          | 0/67349 [00:00<?, ? examples/s]

Tokenizing sst2:   0%|          | 0/872 [00:00<?, ? examples/s]

Tokenizing sst2:   0%|          | 0/1821 [00:00<?, ? examples/s]

Tokenizing mnli:   0%|          | 0/392702 [00:00<?, ? examples/s]

Tokenizing mnli:   0%|          | 0/9815 [00:00<?, ? examples/s]

...

Tokenizing mnli:   0%|          | 0/9796 [00:00<?, ? examples/s]

Tokenizing mnli:   0%|          | 0/9847 [00:00<?, ? examples/s]

Tokenizing qqp:   0%|          | 0/363846 [00:00<?, ? examples/s]

Tokenizing qqp:   0%|          | 0/40430 [00:00<?, ? examples/s]

Tokenizing qqp:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [27]:
print("SST-2 sample structure:")
print(processed_sst2["train"][0])
print("\nMNLI sample structure:")
print(processed_mnli["train"][0])
print("\nQQP sample structure:")
print(processed_qqp["train"][0])

SST-2 sample structure:
{'label': 0, 'idx': 0, 'input_ids': [101, 5342, 2047, 3595, 8496, 2013, 1996, 18643, 3197, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

统一数据格式： 预处理后，每个样本都应该包含：
- input_ids: Token ID 序列。
- attention_mask: 用于告诉模型哪些是真实 Token，哪些是 Padding。
- token_type_ids (可选，但 Bert 常用): 用于区分句子对中的第一个句子和第二个句子。
- labels: 任务对应的标签（需要映射为整数 ID）。

为每个任务分别创建 DataLoader，训练时轮流取 batch   
Huggingface 的 Trainer 会自动把 datasets.Dataset 转成 DataLoader，并自动处理 batch、shuffle、collate 等。你只需要把 train_dataset、eval_dataset 传给 Trainer，不需要手动写 DataLoader。   
但如果你要多任务训练（比如轮流训练不同任务），还是建议自己控制训练循环，或者用 Trainer 的自定义 callback 或者多 Trainer 方案。   

In [28]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [29]:
sst2_loader = DataLoader(
    processed_sst2["train"],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
)
mnli_loader = DataLoader(
    processed_mnli["train"],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
)
qqp_loader = DataLoader(
    processed_qqp["train"],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
)
sst2_eval = DataLoader(
    processed_sst2["validation"],
    batch_size=8,
    collate_fn=data_collator
)
mnli_eval = DataLoader(
    processed_mnli["validation_matched"],
    batch_size=8,
    collate_fn=data_collator
)
qqp_eval = DataLoader(
    processed_qqp["validation"],
    batch_size=8,
    collate_fn=data_collator
)

共享编码器与多任务头设计：
- 共享编码器： 加载预训练的 BERT 模型主体（通常使用 AutoModel，它只输出特征，不带分类头）。这部分参数在所有任务之间共享。
- 多任务头 (Multi-task Heads): 在 BERT 的输出之上，为每个任务添加一个单独的、小的任务特定层（通常是 nn.Linear）。
- SST-2 Head: 输入是 BERT 输出的 [CLS] Token 的特征向量，输出 2 个 logits (正面/负面)。
- QQP Head: 输入是 BERT 输出的 [CLS] Token 的特征向量，输出 2 个 logits (相似/不相似)。
- MNLI Head: 输入是 BERT 输出的 [CLS] Token 的特征向量，输出 3 个 logits (蕴含/矛盾/中性)。
- 模型结构： 你的主模型类应该继承 nn.Module。在 __init__ 中实例化共享的 BERT 编码器和每个任务对应的任务头。在 forward 方法中，根据输入的task_id，决定将 BERT 的输出送给哪个任务头，并返回对应的输出。

In [39]:
class model(nn.Module):
    def __init__(self, bert_model_name, num_labels_dict):
        """
        bert_model_name: str, 预训练的BERT模型名称
        num_labels_dict: dict, 存储数据集名字和输出标签数量的字典，例如 {"mnli": 3, "qqp": 2, "sst2": 2}
        """
        super(model, self).__init__()
        # 共享编码器
        self.bert = AutoModel.from_pretrained(bert_model_name)

        # 多任务头
        self.task_heads = nn.ModuleDict() # 使用 ModuleDict 来存储多个任务头
        for task_name, num_labels in num_labels_dict.items():
             # BERT 的 [CLS] Token 输出维度是 bert.config.hidden_size
            self.task_heads[task_name] = nn.Linear(self.bert.config.hidden_size, num_labels)

    #自己定义模型时，记得把不常规的参数设置默认值None，防止模型实例化后检查forward时报错
    def forward(self, input_ids, attention_mask, token_type_ids=None,task_id=None):
        # 通过共享编码器获取特征表示
        # bert_output.last_hidden_state 的形状是 [batch_size, seq_len, hidden_size]
        # bert_output.pooler_output 的形状是 [batch_size, hidden_size] (通常是 [CLS] 特征经过一个线性层和Tanh)
        # 对于分类任务，通常使用 [CLS] Token 的特征表示
        bert_output = self.bert(input_ids=input_ids,
                               attention_mask=attention_mask,
                               token_type_ids=token_type_ids,
                               return_dict=True)
        cls_embedding = bert_output.last_hidden_state[:, 0, :] # 取 [CLS] Token 的 embedding (第一个 token)
        # 或者使用 pooled_output，取决于bert的实现和你的偏好
        # cls_embedding = bert_output.pooler_output

        # 根据 task_id 将特征送给对应的任务头
        if(task_id == 0):
            task_name = "mnli"
        elif(task_id == 1):
            task_name = "qqp"
        else:
            task_name = "sst2"
        logits = self.task_heads[task_name](cls_embedding)

        return logits # 返回对应任务头的输出 logits

多任务训练：
- 使用梯度累计模拟大批量训练，节省内存
- 使用PCGrad，减少不同任务梯度下降时的冲突

In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaders = { "sst2":sst2_loader, "mnli":mnli_loader, "qqp":qqp_loader }
epochs = 15
accumulation_steps = 4
criterion = nn.CrossEntropyLoss()
model = model("bert-large-uncased", {"mnli": 3, "qqp": 2, "sst2": 2}).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()
best_loss = float("inf")

for epoch in range(epochs):
    running_loss = 0.0
    total_batches = 0
    for i,loader in loaders.items():
        loop = tqdm(loader, leave=False, desc=f"Epoch {epoch+1}/{epochs} - {id}")
        for batch in loop:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            labels = batch["labels"].to(device)
            task_id = batch["task_id"].to(device)
            
            logits = model(
                input_ids,
                attention_mask,
                token_type_ids,
                task_id[0]  # 单一任务
            )
          
            loss = criterion(logits, labels)
            loss = loss / accumulation_steps  # 梯度累积
            loss.backward() 
            if (i + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()
            running_loss += loss.item()
           
            loop.set_postfix(loss=loss.item())
            total_batches += 1
    epoch_loss = running_loss / total_batches
    if (epoch_loss < best_loss):
        best_loss = epoch_loss
        checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),              
                'loss': epoch_loss,
            }
        torch.save(model.state_dict(), f"../model/MultiTaskBert/checkpoint_{epoch}.pth")
    print(f"Epoch {epoch+1}/{epochs} 训练平均损失: {epoch_loss:.4f}")        

AttributeError: 'str' object has no attribute 'size'

评估

In [ ]:
evalers = { "sst2":sst2_eval, "mnli":mnli_eval, "qqp":qqp_eval }
model.eval()

for name,evaler in evalers.items():
    metric_name = evaluate.load("glue", name)
    print(f"开始评估 {name} 数据集")
    for batch in tqdm(evaler):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["labels"].to(device)

        with torch.no_grad():
            outputs = model(
                input_ids,
                attention_mask,
                token_type_ids,
                batch["task_name"][0]  # 单一任务
            )

        logits = outputs
        # model 返回的是 logits（不是一个带 logits 属性的对象），所以这里应该直接用 outputs
        predictions = torch.argmax(logits, dim=-1)
        metric_name.add_batch(predictions=predictions, references=batch["labels"])

    metric_name.compute()
    print(f"{name} 数据集评估完成")
